# Context Managers

Viri:
- [Python Context Managers](https://mattgathu.github.io/python-context-managers/)
- [Python with Context Managers](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/)
- [Context Managers](http://archive.oreilly.com/oreillyschool/courses/Python4/Python4-14.html)
- [Redirecting all kinds of stdout in Python](https://eli.thegreenplace.net/2015/redirecting-all-kinds-of-stdout-in-python/)

## Intro & Overview

A context manager, in Python, is a resource acquisition and release mechanism that prevents resource leak and ensures startup and cleanup (exit) actions are always done.

A resource is basically a computing component with limited availability e.g. files, network sockets etc. The act of refusing to release a resource when a process has finished using it is known as a resource leak. An example would be leaving a file open after writing into it, thereby making it impossible for other processes to acquire it.

The main motivation behind context managers is to ease resource management by providing support for resource acquisition and release. This introduces to several advantages:
- eliminates the need of repeating resource acquisition/release code fragments: the DRY principle.
- prevent errors arounds resource management.
- eases code refactoring: this is a consequence of DRY principle.
- makes resource cleanup easier: by guarranteeing startup (entry) and cleanup (exit) actions.

So what’s the with statement good for? It helps simplify some common
resource management patterns by abstracting their functionality
and allowing them to be factored out and reused.

    with something_that_returns_a_context_manager() as my_resource:
        do_something(my_resource)
        ...
        print('done using my_resource')

## Context Manager Example (Opening Files)

A good way to see this feature used effectively is by looking at examples
in the Python standard library. The built-in open() function provides
us with an excellent use case:

In [1]:
with open('data/hello.txt', 'w') as f:
    f.write('hello, world!')

In [2]:
f

<_io.TextIOWrapper name='data/hello.txt' mode='w' encoding='UTF-8'>

In [3]:
f.closed

True

In [4]:
f.encoding

'UTF-8'

In [5]:
f.write('test')
# But you can’t perform I/O with fp because at the end of the with block, the
#TextIOWrapper.__exit__ method is called and closes the file.

ValueError: I/O operation on closed file.

Crucially, the variable only exists within the indented block below the with statement. Think of with as creating a mini-function: we can use the variable freely in the indented portion, but once that block ends, the variable goes out of scope. When the variable goes out of scope, it automatically calls a special method that contains the code to clean up the resource.

> Perhaps the most common (and important) use of context managers is to properly manage resources. In fact, that's the reason we use a context manager when reading from a file. The act of opening a file consumes a resource (called a file descriptor), and this resource is limited by your OS. That is to say, there are a maximum number of files a process can have open at one time. Joking aside, what is a "file descriptor" and what does it mean to "leak" one? Well, when you open a file, the operating system assigns an integer to the open file, allowing it to essentially give you a handle to the open file rather than direct access to the underlying file itself. This is beneficial for a variety of reasons, including being able to pass references to files between processes and to maintain a certain level of security enforced by the kernel.

Opening files using the with statement is generally recommended because
it ensures that open file descriptors are closed automatically after
program execution leaves the context of the with statement. Internally,
the above code sample translates to something like this:

In [2]:
f = open('hello.txt', 'w')
try:
    f.write('hello, world')
finally:
    f.close()

> In other languages, developers are forced to use try...except...finally every time they work with a file (or any other type of resource that needs to be closed, like sockets or database connections). Luckily, Python loves us and gives us a simple way to make sure all resources we use are properly cleaned up, regardless of if the code returns or an exception is thrown: context managers.

You can already tell that this is quite a bit more verbose. Note that
the try...finally statement is significant. It wouldn’t be enough to
just write something like this:

In [3]:
f = open('hello.txt', 'w')
f.write('hello, world')
f.close()

This implementation won’t guarantee the file is closed if there’s an exception
during the f.write() call—and therefore our program might
leak a file descriptor. That’s why the with statement is so useful. It
makes properly acquiring and releasing resources a breeze.

Given that context managers are so helpful, they were added to the Standard Library in a number of places. Lock objects in threading are context managers, as are zipfile.ZipFiles. subprocess.Popen, tarfile.TarFile, telnetlib.Telnet, pathlib.Path... the list goes on and on. Essentially, any object that needs to have close called on it after use is (or should be) a context manager.

## Creating a Context Manager class

Now, there’s nothing special or magical about the open() function or
the threading.Lock class and the fact that they can be used with a
with statement. You can provide the same functionality in your own
classes and functions by implementing so-called context managers.

What’s a context manager? It’s a simple “protocol” (or interface) that
your object needs to follow in order to support the with statement.
Basically, all you need to do is add `__enter__` and `__exit__` methods
to an object if you want it to function as a context manager. Python
will call these two methods at the appropriate times in the resource
management cycle.

Let’s take a look at what this would look like in practical terms. Here’s
what a simple implementation of the open() context manager might
look like:

In [6]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

The interpreter calls the `__enter__` method with no arguments—beyond the implicit
self. The three arguments passed to `__exit__` are:
- exc_type
    - The exception class (e.g., ZeroDivisionError).
- exc_value
    - The exception instance. Sometimes, parameters passed to the exception constructor such as the error message—can be found in exc_value.args.
- traceback
    - A traceback object.

Our ManagedFile class follows the context manager protocol and now
supports the with statement, just like the original open() example
did:

In [7]:
with ManagedFile('data/hello_managed.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

Python calls `__enter__` when execution enters the context of the
with statement and it’s time to acquire the resource. When execution
leaves the context again, Python calls `__exit__` to free up the
resource.

## Understanding a Context Manager class

The with statement sets up a temporary context and reliably tears it down, under the
control of a context manager object. This prevents errors and reduces boilerplate code,
making APIs at the same time safer and easier to use. Python programmers are finding
lots of uses for with blocks beyond automatic file closing

Context manager objects exist to control a with statement, just like iterators exist to
control a for statement.

The with statement was designed to simplify the try/finally pattern, which guarantees
that some operation is performed after a block of code, even if the block is aborted
because of an exception, a return or sys.exit() call. The code in the finally clause
usually releases a critical resource or restores some previous state that was temporarily
changed.

The context manager protocol consists of the `__enter__` and `__exit__` methods. At the
start of the with,` __enter__` is invoked on the context manager object. The role of the
finally clause is played by a call to `__exit__` on the context manager object at the end
of the with block.

The context manager object is the result of evaluating the expression after with, but the value bound to the
target variable (in the as clause) is the result of calling `__enter__` on the context manager
object.

But the `__enter__` method may also return some other object instead of the context manager.

When control flow exits the with block in any way, the `__exit__` method is invoked on
the context manager object, not on whatever is returned by `__enter__`.

The as clause of the with statement is optional. In the case of open, you’ll always need
it to get a reference to the file, but some context managers return None because they
have no useful object to give back to the user.

Example shows the operation of a perfectly frivolous context manager designed to
highlight the distinction between the context manager and the object returned by its
`__enter__` method.

In [10]:
from skripte.mirror import LookingGlass

with LookingGlass() as what: #1
    print('Danes je lep dan.') #2,3 
    print(what)

.nad pel ej senaD
YKCOWREBBAJ


In [11]:
what #4

'JABBERWOCKY'

In [12]:
print('Danes je lep dan.') #5

Danes je lep dan.


1. The context manager is an instance of LookingGlass; Python calls `__enter__` on the context manager and the result is bound to what.
2. Print a str, then the value of the target variable what.
3. The output of each print comes out backward.
4. Now the with block is over. We can see that the value returned by `__enter__`, held in what, is the string 'JABBERWOCKY'.
5. Program output is no longer backward.

Implementation of LookingGlass:

In [1]:
#mirror.py
class LookingGlass:
    def __enter__(self): #1
        import sys
        self.original_write = sys.stdout.write #2
        sys.stdout.write = self.reverse_write #3
        return 'JABBERWOCKY' #4

    def reverse_write(self, text): #5
        self.original_write(text[::-1])
    
    def __exit__(self, exc_type, exc_value, traceback): #6
        import sys #7
        sys.stdout.write = self.original_write #8
        if exc_type is ZeroDivisionError: #9
            print('Please DO NOT divide by zero!')
            return True #10
        #11

1. Python invokes `__enter__` with no arguments besides self.
2. Hold the original sys.stdout.write method in an instance attribute for later use.
3. Monkey-patch sys.stdout.write, replacing it with our own method.
4. Return the 'JABBERWOCKY' string just so we have something to put in the target variable what.
5. Our replacement to sys.stdout.write reverses the text argument and calls the original implementation.
6. Python calls `__exit__` with None, None, None if all went well; if an exception is raised, the three arguments get the exception data, as described next.
7. It’s cheap to import modules again because Python caches them.
8. Restore the original method to sys.stdout.write.
9. If the exception is not None and its type is ZeroDivisionError, print a message…
10. …and return True to tell the interpreter that the exception was handled.
11. If `__exit__` returns None or anything but True, any exception raised in the with block will be propagated.

> When real applications take over standard output, they often want
to replace sys.stdout with another file-like object for a while, then
switch back to the original. The contextlib.redirect_stdout
context manager does exactly that: just pass it the file-like object
that will stand in for sys.stdout.

The interpreter calls the `__enter__` method with no arguments—beyond the implicit self. The three arguments passed to `__exit__` are:
- `exc_type`: The exception class (e.g., ZeroDivisionError).
- `exc_value`: The exception instance. Sometimes, parameters passed to the exception constructor—such as the error message—can be found in exc_value.args.
- `traceback`: A traceback object.

In [26]:
#mirror.py
class LookingGlass2:
    def __enter__(self): #1
        import sys
        self.original_write = sys.stdout.write #2
        sys.stdout.write = self.reverse_write #3
        return 'JABBERWOCKY' #4

    def reverse_write(self, text): #5
        self.original_write(text[::-1])
    
    def __exit__(self, exc_type, exc_value, traceback): #6
        import sys #7
        sys.stdout.write = self.original_write #8
        print('exc_type: ', exc_type)
        print('exc_value: ', exc_value)
        print('traceback: ', traceback)
        if exc_type is ZeroDivisionError: #9
            print('Please DO NOT divide by zero!')
            return True #10
        #11

In [27]:
with LookingGlass2() as what:
    print('Danes je lep dan.')

.nad pel ej senaD
exc_type:  None
exc_value:  None
traceback:  None


In [28]:
with LookingGlass2() as what:
    print('Danes je lep dan.')
    print(8/0)

.nad pel ej senaD
exc_type:  <class 'ZeroDivisionError'>
exc_value:  division by zero
traceback:  <traceback object at 0x7f11e9fe6748>
Please DO NOT divide by zero!


In [30]:
a = {1: 'asas'}

with LookingGlass2() as what:
    print('Danes je lep dan.')
    print(a[2])

.nad pel ej senaD
exc_type:  <class 'KeyError'>
exc_value:  2
traceback:  <traceback object at 0x7f11e9fe6448>


KeyError: 2

For a detailed look at how a context manager works, see next Example, where Looking
Glass is used outside of a with block, so we can manually call its `__enter__` and
`__exit__` methods.

In [43]:
from skripte.mirror import LookingGlass
manager = LookingGlass() #1

In [44]:
manager

In [45]:
monster = manager.__enter__() #2

In [46]:
print(monster == 'JABBERWOCKY') #3

eurT


In [47]:
print(monster)

YKCOWREBBAJ


In [48]:
print(manager)

>8d6bde9e11f7x0 ta tcejbo ssalGgnikooL.rorrim.etpirks<


In [49]:
manager.__exit__(None, None, None) #4

In [50]:
print(monster)

JABBERWOCKY


1. Instantiate and inspect the manager instance.
2. Call the context manager `__enter__()` method and store result in monster.
3. Monster is the string 'JABBERWOCKY'. The True identifier appears reversed because all output via stdout goes through the write method we patched in `__enter__`.
4. Call `manager.__exit__` to restore previous stdout.write.

Context managers are a fairly novel feature and slowly but surely the Python community
is finding new, creative uses for them. Some examples from the standard library are:
-  Managing transactions in the sqlite3 module; see “12.6.7.3. Using the connection
as a context manager”.
- Holding locks, conditions, and semaphores in threading code; see “17.1.10. Using
locks, conditions, and semaphores in the with statement”.
- Setting up environments for arithmetic operations with Decimal objects; see the
decimal.localcontext documentation.
- Applying temporary patches to objects for testing; see the unittest.mock.patch
function.

## The contextlib Utilities

[contextlib — Utilities for with-statement contexts](https://docs.python.org/3/library/contextlib.html)

Before rolling your own context manager classes, take a look at “29.6 contextlib —
Utilities for with-statement contexts” in The Python Standard Library. Besides the already
mentioned redirect_stdout, the contextlib module includes classes and other
functions that are more widely applicable:
    
- `closing`: A function to build context managers out of objects that provide a close() method but don’t implement the `__enter__/__exit__` protocol.
- `suppress`: A context manager to temporarily ignore specified exceptions.
- `@contextmanager`:  A decorator that lets you build a context manager from a simple generator function, instead of creating a class and implementing the protocol. 
- `ContextDecorator`: A base class for defining class-based context managers that can also be used as function decorators, running the entire function within a managed context.
- `ExitStack`:  A context manager that lets you enter a variable number of context managers. When the with block ends, ExitStack calls the stacked context managers’ `__exit__` methods in LIFO order (last entered, first exited). Use this class when you don’t know beforehand how many context managers you need to enter in your with block; for example, when opening all files from an arbitrary list of files at the same time.

The most widely used of these utilities is surely the @contextmanager decorator, so it
deserves more attention. That decorator is also intriguing because it shows a use for the
yield statement unrelated to iteration. This paves the way to the concept of a coroutine,
the theme of the next chapter.

## Using @contextmanager

The @contextmanager decorator reduces the boilerplate of creating a context manager:
instead of writing a whole class with `__enter__/__exit__` methods, you just implement
a generator with a single yield that should produce whatever you want the `__enter__` method to return.

In a generator decorated with @contextmanager, yield is used to split the body of the
function in two parts: everything before the yield will be executed at the beginning of
the while block when the interpreter calls `__enter__`; the code after yield will run
when `__exit__` is called at the end of the block.

Here is an example. Example replaces the LookingGlass class with a generator function.

In [1]:
import contextlib

@contextlib.contextmanager #1
def looking_glass():
    import sys
    original_write = sys.stdout.write #2

    def reverse_write(text): #3
        original_write(text[::-1])

    sys.stdout.write = reverse_write #4
    yield 'JABBERWOCKY' #5
    sys.stdout.write = original_write #6

1. Apply the contextmanager decorator.
2. Preserve original sys.stdout.write method.
3. Define custom reverse_write function; original_write will be available in the closure.
4. Replace sys.stdout.write with reverse_write.
5. Yield the value that will be bound to the target variable in the as clause of the with statement. This function pauses at this point while the body of the with executes.
6. When control exits the with block in any way, execution continues after the yield; here the original sys.stdout.write is restored.

In [2]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [53]:
what

'JABBERWOCKY'

Essentially the contextlib.contextmanager decorator wraps the function in a class
that implements the `__enter__` and `__exit__` methods.
The `__enter__` method of that class:
1. Invokes the generator function and holds on to the generator object—let’s call it gen.
2. Calls next(gen) to make it run to the yield keyword.
3. Returns the value yielded by next(gen), so it can be bound to a target variable in
the with/as form.

When the with block terminates, the `__exit__` method:
1. Checks an exception was passed as exc_type; if so, gen.throw(exception) is invoked,
causing the exception to be raised in the yield line inside the generator
function body.
2. Otherwise, next(gen) is called, resuming the execution of the generator function
body after the yield.

Our example has a serious flaw: if an exception is raised in the body of the with block,
the Python interpreter will catch it and raise it again in the yield expression inside
looking_glass. But there is no error handling there, so the looking_glass function will 
abort without ever restoring the original sys.stdout.write method, leaving the
system in an invalid state.

Next example adds special handling of the ZeroDivisionError exception, making it functionally equivalent to the class-based Example.

In [54]:
import contextlib
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = '' #1
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError: #2 
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write #3
        if msg:
            print(msg) #4

1. Create a variable for a possible error message; this is the first change in relation to Example 15-5.
2. Handle ZeroDivisionError by setting an error message.
3. Undo monkey-patching of sys.stdout.write.
4. Display error message, if it was set.

Recall that the `__exit__` method tells the interpreter that it has handled the exception
by returning True; in that case, the interpreter suppresses the exception. On the other
hand, if `__exit__` does not explicitly return a value, the interpreter gets the usual
None, and propagates the exception. With @contextmanager, the default behavior is
inverted: the `__exit__` method provided by the decorator assumes any exception sent
into the generator is handled and should be suppressed. You must explicitly re-raise an
exception in the decorated function if you don’t want @contextmanager to suppress it.

> Having a try/finally (or a with block) around the yield is an
unavoidable price of using @contextmanager, because you never
know what the users of your context manager are going to do inside
their with block.

In [81]:
# primer open funkcija
from contextlib import contextmanager

@contextmanager
def file_open(path):
    try:
        f_obj = open(path, 'w')
        yield f_obj
    except OSError:
        print("We had an error!")
    finally:
        print('Closing file')
        f_obj.close()

if __name__ == '__main__':
    with file_open('data/test.txt') as fobj:
        fobj.write('Testing context managers')

Closing file


An interesting real-life example of @contextmanager outside of the standard library is
[Martijn Pieters’ in-place file rewriting context manager](https://www.zopatista.com/python/2013/11/26/inplace-file-rewriting/).

## contextlib.redirect_stdout / redirect_stderr

A common task in Python (especially while testing or debugging) is to redirect sys.stdout to a stream or a file while executing some piece of code.

The simplest case arises when the underlying Python code writes to stdout, whether by calling print, sys.stdout.write or some equivalent method. If the code you have does all its printing from Python, redirection is very easy. With Python 3.4 we even have a built-in tool in the standard library for this purpose - contextlib.redirect_stdout. Here's how to use it:

In [58]:
import io
from contextlib import redirect_stdout

f = io.StringIO()

with redirect_stdout(f):
    print('foobar')
    print(12)

In [60]:
f.getvalue()

'foobar\n12\n'

In [65]:
f.seek(0)
print(f.read())

foobar
12



When this code runs, the actual print calls within the with block don't emit anything to the screen, and you'll see their output captured by in the stream f. Incidentally, note how perfect the with statement is for this goal - everything within the block gets redirected; once the block is done, things are cleaned up for you and redirection stops.

Redirect_stdout is really easy to implement on your own. I'll change its name slightly to avoid confusion:

In [68]:
from contextlib import contextmanager
import sys

@contextmanager
def stdout_redirector(stream):
    old_stdout = sys.stdout
    sys.stdout = stream
    try:
        yield
    finally:
        sys.stdout = old_stdout

In [72]:
f = io.StringIO()
with stdout_redirector(f):
    print('test123')
    print(1245)

In [73]:
f.seek(0)
print(f.read())

test123
1245



## Primer: SQLite context maneger

Rather than rewrite Python’s open method here, we’ll create a context manager that can create a SQLite database connection and close it when it’s done. Here’s a simple example:

In [79]:
import sqlite3


class DataConn:
    """"""
    def __init__(self, db_name):
        """Constructor"""
        self.db_name = db_name

    def __enter__(self):
        """
        Open the database connection
        """
        self.conn = sqlite3.connect(self.db_name)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Close the connection
        """
        self.conn.close()
        if exc_val:
            raise

if __name__ == '__main__':
    db = 'data/test.db'
    with DataConn(db) as conn:
        cursor = conn.cursor()
        # Create table
        cursor.execute('''CREATE TABLE stocks
                     (date text, trans text, symbol text, qty real, price real)''')
        # Insert a row of data
        cursor.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
        # Save (commit) the changes
        conn.commit()
        cursor.execute("select * from stocks")
        print(cursor.fetchall())

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]


In the code above, we created a class that takes a path to a SQLite database file. The `__enter__` method executes automatically where it creates and returns the database connection object. Now that we have that, we can create a cursor and write to the database or query it. When we exit the with statement, it causes the `__exit__` method to execute and that closes the connection.